In [1]:
from personal_twitter_data import TweetLoader
from mongo_data import MongoStore

In [2]:
loader = TweetLoader()
loader.extract_tweets(count=5)

In [3]:
loader.get_loaded_tweets_as_json()

[{'created_at': 'Thu Dec 01 01:51:30 +0000 2022',
  'id': 1598132652296802305,
  'id_str': '1598132652296802305',
  'text': '"HEART IS IN SPACE": @jesseprimetime reacts to Vice President Harris\'s meeting with French President Emmanuel Macro… https://t.co/XCZheoR46L',
  'truncated': True,
  'entities': {'hashtags': [],
   'symbols': [],
   'user_mentions': [{'screen_name': 'jesseprimetime',
     'name': 'Jesse Watters Primetime',
     'id': 819334191624101888,
     'id_str': '819334191624101888',
     'indices': [21, 36]}],
   'urls': [{'url': 'https://t.co/XCZheoR46L',
     'expanded_url': 'https://twitter.com/i/web/status/1598132652296802305',
     'display_url': 'twitter.com/i/web/status/1…',
     'indices': [117, 140]}]},
  'source': '<a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a>',
  'in_reply_to_status_id': None,
  'in_reply_to_status_id_str': None,
  'in_reply_to_user_id': None,
  'in_reply_to_user_id_str': None,
  'in_reply_to_screen_name': None,
  'use

In [4]:
for tweet in loader.tweets:
    print(tweet.author.name)
    # print(tweet._json)

Fox News
Forbes
Boston Celtics
Fox News
NBA


In [2]:
store = MongoStore("twitter_dashboard_db", "home_timeline")

In [6]:
store.save_tweets(loader.tweets)

1. FoxNews
2. Forbes
3. celtics
4. FoxNews
5. NBA


In [10]:
data = store.load_data()
len(data)

5

In [30]:
data = store.load_data(latest=True)

for user_data in data:
    print(user_data["user"])
    print(len(user_data["tweets"]))
    for tweet in user_data["tweets"]:
        print(tweet["created_at"])

business
3
Thu Dec 01 01:18:03 +0000 2022
Sat Nov 26 00:30:33 +0000 2022
Sat Nov 26 00:36:04 +0000 2022
celtics
2
Thu Dec 01 01:50:04 +0000 2022
Sat Nov 26 00:33:36 +0000 2022
Forbes
5
Thu Dec 01 01:50:05 +0000 2022
Thu Dec 01 01:15:04 +0000 2022
Thu Dec 01 01:15:31 +0000 2022
Sat Nov 26 00:30:08 +0000 2022
Sat Nov 26 00:30:19 +0000 2022
NBA
2
Thu Dec 01 01:50:04 +0000 2022
Thu Dec 01 01:18:16 +0000 2022
FoxNews
3
Thu Dec 01 01:50:04 +0000 2022
Thu Dec 01 01:51:30 +0000 2022
Thu Dec 01 01:15:03 +0000 2022
